In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [6]:
# read CSV file into dataframe
wine_df = pd.read_csv("winedata.csv",index_col=0)
# wine_df.head()

FileNotFoundError: File b'winedata.csv' does not exist

In [3]:
# drop unneeded columns 
wine_df.drop(["country", "designation","points", "price", "province", "region_1", "taster_name", "taster_twitter_handle","region_2", "title","winery" ], axis = 1, inplace = True)

In [ ]:
wine_df.head()

In [ ]:
wine_df.count()

In [4]:
# normalize columns by removing special characters
cols = wine_df.select_dtypes(include=[np.object]).columns
wine_df[cols] = wine_df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
# wine_df

In [5]:
# get top 20 wine varietals from counts
top20 = wine_df['variety'].value_counts().nlargest(20)

In [ ]:
top20

In [6]:
# only look at rows with top 20 varietals
wine_df_final =wine_df[wine_df['variety'].isin(top20.index)]

In [7]:
wine_df_final.count()

description    93914
variety        93914
dtype: int64

In [ ]:
wine_df_final.variety.unique()

In [ ]:
wine_df_final.head()

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer # term 


In [9]:
#normalize labels for variety
le = LabelEncoder()
y = le.fit_transform(wine_df_final["variety"])

In [10]:
y

array([18,  8,  6, ..., 11,  7,  6])

In [11]:
le.classes_

array(['Bordeaux-style Red Blend', 'Cabernet Sauvignon', 'Chardonnay',
       'Malbec', 'Merlot', 'Nebbiolo', 'Pinot Gris', 'Pinot Noir',
       'Portuguese Red', 'Red Blend', 'Rhone-style Red Blend', 'Riesling',
       'Rose', 'Sangiovese', 'Sauvignon Blanc', 'Sparkling Blend', 'Syrah',
       'Tempranillo', 'White Blend', 'Zinfandel'], dtype=object)

In [12]:
stopwords = text.ENGLISH_STOP_WORDS

In [13]:
unique_varietals = wine_df_final.variety.unique()
unique_varietals

array(['White Blend', 'Portuguese Red', 'Pinot Gris', 'Riesling',
       'Pinot Noir', 'Cabernet Sauvignon', 'Chardonnay', 'Malbec',
       'Red Blend', 'Merlot', 'Sauvignon Blanc', 'Sangiovese',
       'Bordeaux-style Red Blend', 'Rose', 'Zinfandel', 'Syrah',
       'Nebbiolo', 'Rhone-style Red Blend', 'Sparkling Blend',
       'Tempranillo'], dtype=object)

In [14]:
wine_stopwords = []
for variety in wine_df_final['variety'].unique():
    for word in variety.split(' '):
        wine_stopwords.append(word.lower())
# wine_stop_words = pd.Series(data=wine_stop_words).unique()

In [15]:
wine_stopwords

['white',
 'blend',
 'portuguese',
 'red',
 'pinot',
 'gris',
 'riesling',
 'pinot',
 'noir',
 'cabernet',
 'sauvignon',
 'chardonnay',
 'malbec',
 'red',
 'blend',
 'merlot',
 'sauvignon',
 'blanc',
 'sangiovese',
 'bordeaux-style',
 'red',
 'blend',
 'rose',
 'zinfandel',
 'syrah',
 'nebbiolo',
 'rhone-style',
 'red',
 'blend',
 'sparkling',
 'blend',
 'tempranillo']

In [ ]:
# more_stopwords = []
# for unique in unique_varietals:
#     for word in unique.split(' '):
#         yy = unique.lower()
#         more_stopwords.append(yy)

In [ ]:
# more_stopwords

In [16]:
wine_stopwords.append(stopwords)

In [17]:
vector2 = TfidfVectorizer(stop_words= wine_stopwords)
X = vector2.fit_transform(wine_df_final["description"])

In [18]:
X

<93914x29607 sparse matrix of type '<class 'numpy.float64'>'
	with 3173898 stored elements in Compressed Sparse Row format>

In [ ]:
len(X.toarray()[0])

In [19]:
#split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=72, stratify = y)

In [ ]:
target = test_df['stars']

In [4]:
X_train.shape

NameError: name 'X_train' is not defined

In [37]:
# Support vector machine classifier
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [38]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=le.classes_))

                          precision    recall  f1-score   support

Bordeaux-style Red Blend       0.69      0.77      0.72      2074
      Cabernet Sauvignon       0.57      0.66      0.61      2842
              Chardonnay       0.74      0.86      0.80      3526
                  Malbec       0.45      0.36      0.40       796
                  Merlot       0.33      0.16      0.22       931
                Nebbiolo       0.76      0.80      0.78       841
              Pinot Gris       0.60      0.34      0.44       436
              Pinot Noir       0.69      0.79      0.74      3982
          Portuguese Red       0.72      0.70      0.71       740
               Red Blend       0.63      0.59      0.61      2684
   Rhone-style Red Blend       0.70      0.60      0.64       441
                Riesling       0.77      0.82      0.79      1557
                    Rose       0.74      0.71      0.73      1069
              Sangiovese       0.60      0.49      0.54       812
         

In [22]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.664


In [39]:
test_data = ['fruity', 'oak', 'medium-bodied']
test_prediction = vector2.transform(test_data)

In [40]:
model.predict(test_prediction)

array([19,  2,  2])

In [41]:
le.classes_[model.predict(test_prediction)]

array(['Zinfandel', 'Chardonnay', 'Chardonnay'], dtype=object)

In [34]:
svm_test = joblib.load('svm_wine.pkl')

In [28]:
from sklearn.externals import joblib
joblib.dump(model, 'svm_wine.pkl')

['svm_wine.pkl']

In [ ]:
from sklearn.externals import joblib
joblib.dump(model, 'svm_wine.pkl')